In [ ]:
import pandas as pd
from os import path
from datetime import datetime as dt
from datetime import timedelta
import numpy as np

# Prepare data

In [ ]:
%%time
# get dataframe

fp = path.abspath('../dash_plotly_code/data/raw')

data = pd.read_csv(path.abspath('../dash_plotly_code/data/data.csv'))
data['Time'] = [dt.strptime(i, '%m/%d/%Y %H:%M:%S') for i in data['Time']]

coordinates = pd.read_csv(path.abspath('../dash_plotly_code/data/cities_coord_time_alt.csv'))

In [ ]:
data.nunique()

In [ ]:
print(data.shape)
data.isnull().sum()

In [ ]:
print(coordinates.shape)
coordinates.isnull().sum()

In [ ]:
# data = data.dropna()

In [ ]:
# (data.Dose_Rate==0).sum()

In [ ]:
# data.Dose_Rate = data.Dose_Rate.replace(0, np.nan)

In [ ]:
# (data.Dose_Rate==0).sum()

In [ ]:
coordinates.head()

In [ ]:
DF = pd.merge(data, coordinates, on=['City', 'State'], how='inner')

In [ ]:
print(DF.shape)
DF.isnull().sum()

In [ ]:
# drop rows where both are nan
DF.dropna(subset=['Dose_Rate', 'Gamma_Count'], how='all', inplace=True)

In [ ]:
print(DF.shape)
DF.isnull().sum()

In [ ]:
DF.nunique()

In [ ]:
%%time
DF['Time'] = [i+timedelta(hours=int(j)) for i,j in zip(DF['Time'], DF['UTC'])] 

In [ ]:
DF.head()

# Exploratory Data Analysis

In [ ]:
# import plotly.express as px
# fig = px.histogram(DF, x="Dose_Rate")
# fig.show()

In [ ]:
# fig = px.histogram(DF, x="Gamma_Count")
# fig.show()

# Map plot

In [ ]:
df1 = DF[[i.year == 2020 and
         i.month == 5 and
         i.day == 12 
         for i in DF.Time]]

In [ ]:
df1['hour'] = [i.hour for i in df1.Time]

In [ ]:
df1.head()

In [ ]:
import plotly.express as px

metric = 'Gamma_Count'
df1_temp = df1.dropna(subset=[metric])

fig = px.scatter_geo(df1_temp, color=metric, size=metric,
                     lat = 'lat', lon = 'lng', animation_frame ='hour')

fig.update_layout(
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

# Time Series

In [ ]:
df2 = DF[[i in ['AK', 'CA'] for i in DF.State]]

In [ ]:
fig = px.line(df2, x='Time', y='Gamma_Count', color='City', hover_name="State")
fig.show()

In [ ]:
df22 = df2.set_index(['Time','State', 'City'])
df22 = df22.rolling(360).mean()
df22.reset_index(inplace=True)

In [ ]:
fig = px.line(df22, x='Time', y='Gamma_Count', color='City', hover_name="City")
fig.show()

# treemap

In [ ]:
df1.head()

In [ ]:
df3 = df1.groupby(['State', 'City', 'Elevation'], as_index=False)['Dose_Rate'].median()

In [ ]:
df3.head()

In [ ]:
import numpy as np
fig = px.treemap(df3, path=[px.Constant('US'), 'State', 'City'], values='Dose_Rate',
                  color='Dose_Rate', hover_data=['Dose_Rate'])
fig.show()

# Metric vs. Elevation

In [ ]:
df4 = DF.groupby(['City','State','Elevation'])['Gamma_Count', 'Dose_Rate'].median()
df4 = df4.reset_index()

In [ ]:
df4.head()

In [ ]:
import plotly.express as px

fig = px.scatter(df4, x="Elevation", y="Gamma_Count", marginal_y="box",
           marginal_x="box", template="simple_white", hover_data=['City'])
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(df4, x="Gamma_Count", y="Dose_Rate", marginal_y="box",
           marginal_x="box", template="simple_white", hover_data=['City'])
fig.show()

# Bar plots

In [ ]:
metric = 'Gamma_Count'
df1_temp = df1.dropna(subset=[metric]).groupby(['City'])[metric].median().sort_values(ascending=True).reset_index()

In [ ]:

fig = px.bar(df1_temp, x=metric, y="City", orientation='h')
fig.show() 